# Tutorial 06: Networks from OpenStreetMap

In this tutorial, we discuss how networks that have been imported from OpenStreetMap can be integrated and run in Flow. This will all be presented via the Bay Bridge network, seen in the figure below. Networks from OpenStreetMap are commonly used in many traffic simulators for the purposes of replicating traffic in realistic traffic geometries. This is true in both SUMO and Aimsun (which are both supported in Flow), with each supporting several techniques for importing such network files. This process is further simplified and abstracted in Flow, with users simply required to specify the path to the osm file in order to simulate traffic in the network.


<img src="img/bay_bridge_osm.png" width=750>
<center> **Figure 1**: Snapshot of the Bay Bridge from OpenStreetMap </center>

Before we begin, let us import all relevant Flow parameters as we have done for previous tutorials. If you are unfamiliar with these parameters, you are encouraged to review tutorial 1.

In [8]:
# the TestEnv environment is used to simply simulate the network
from flow.envs import TestEnv

# the Experiment class is used for running simulations
from flow.core.experiment import Experiment

# all other imports are standard
from flow.core.params import VehicleParams
from flow.core.params import NetParams
from flow.core.params import InitialConfig
from flow.core.params import EnvParams
from flow.core.params import SumoParams

## 1. Running a Default Simulation

In order to create a network object in Flow with network features depicted from OpenStreetMap, we will use the base `Network` class. This class can sufficiently support the generation of any .osm file.

In [9]:
from flow.networks import Network

In order to recreate the network features of a specific osm file, the path to the osm file must be specified in `NetParams`. For this example, we will use an osm file extracted from the section of the Bay Bridge as depicted in Figure 1.

In order to specify the path to the osm file, simply fill in the `osm_path` attribute with the path to the .osm file as follows:

In [10]:
net_params = NetParams(
    osm_path='networks/milan.osm'
)

Next, we create all other parameters as we have in tutorials 1 and 2. For this example, we will assume a total of 1000 are uniformly spread across the Bay Bridge. Once again, if the choice of parameters is unclear, you are encouraged to review Tutorial 1.

In [11]:
# vehicles dynamics models
from flow.controllers import IDMController, ContinuousRouter
from flow.controllers.routing_controllers import MinicityRouter

# create the remainding parameters
env_params = EnvParams()
sim_params = SumoParams(render=True)
initial_config = InitialConfig()
vehicles = VehicleParams()
vehicles.add('human', num_vehicles=100, routing_controller = (MinicityRouter, {}))

We are finally ready to test our network in simulation. In order to do so, we create an `Experiment` object and run the simulation for a number of steps. This is done in the cell below.

In [12]:


flow_params = dict(
    exp_tag='bay_bridge',
    env_name=TestEnv,
    network=Network,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
)

# number of time steps
flow_params['env'].horizon = 1000
exp = Experiment(flow_params)

#print(vars(Network))
#print(Network.connections)
print(exp.env.network.connections)

No routes specified, defaulting to single edge routes.
None


In [13]:
# run the sumo simulation
_ = exp.run(1)

Round 0, return: 0
Average, std returns: 0.0, 0.0
Average, std velocities: 6.911240639249216, 0.0
Average, std outflows: 3060.0, 0.0
Total time: 18.331687211990356
steps/second: 111.88341251873061


## 2. Customizing the Network

While the above example does allow you to view the network within Flow, the simulation is limited for two reasons. For one, vehicles are placed on all edges within the network; if we wished to simulate traffic solely on the on the bridge and do not care about the artireols, for instance, this would result in unnecessary computational burdens. Next, as you may have noticed if you ran the above example to completion, routes in the base network class are defaulted to consist of the vehicles' current edges only, meaning that vehicles exit the network as soon as they reach the end of the edge they are originated on. In the next subsections, we discuss how the network can be modified to resolve these issues.

### 2.1 Specifying Traversable Edges

In order to limit the edges vehicles are placed on to the road sections edges corresponding to the westbound Bay Bridge, we define an `EDGES_DISTRIBUTION` variable. This variable specifies the names of the edges within the network that vehicles are permitted to originated in, and is assigned to the network via the `edges_distribution` component of the `InitialConfig` input parameter, as seen in the code snippet below. Note that the names of the edges can be identified from the .osm file or by right clicking on specific edges from the SUMO gui (see the figure below).

<img src="img/osm_edge_name.png" width=600>
<center> **Figure 2**: Name of an edge from SUMO </center>

In [14]:
# we define an EDGES_DISTRIBUTION variable with the edges within 
# the westbound Bay Bridge 
# EDGES_DISTRIBUTION = [
#     "11197898",
#     "123741311", 
#     "123741303",
#     "90077193#0",
#     "90077193#1", 
#     "340686922", 
#     "236348366", 
#     "340686911#0",
#     "340686911#1",
#     "340686911#2",
#     "340686911#3",
#     "236348361", 
#     "236348360#0", 
#     "236348360#1"
# ]

# # the above variable is added to initial_config
 new_initial_config = InitialConfig(
#     edges_distribution=EDGES_DISTRIBUTION
 )

IndentationError: unexpected indent (<ipython-input-14-637cb53b8fd9>, line 21)

### 2.2 Creating Custom Routes

Next, we choose to specify the routes of vehicles so that they can traverse the entire Bay Bridge, instead of the only the edge they are currently on. In order to this, we create a new network class that inherits all its properties from `Network` and simply redefine the routes by modifying the `specify_routes` variable. This method was originally introduced in Tutorial 07: Creating Custom Network. The new network class looks as follows:

In [ ]:
# we create a new network class to specify the expected routes
class BayBridgeOSMNetwork(Network):

    def specify_routes(self, net_params):
        return {
            #{'-100183083#0': [(['-100183083#0'], 1)], '-100183083#1': [(['-100183083#1'], 1)], '-120458050': [(['-120458050'], 1)], '-120458052': [(['-120458052'], 1)], '-120458056': [(['-120458056'], 1)], '-134397031#0': [(['-134397031#0'], 1)], '-134397031#1': [(['-134397031#1'], 1)], '-134397031#2': [(['-134397031#2'], 1)], '-142387176#0': [(['-142387176#0'], 1)], '-142387176#1': [(['-142387176#1'], 1)], '-145014444#0': [(['-145014444#0'], 1)], '-145014444#1': [(['-145014444#1'], 1)], '-145014444#2': [(['-145014444#2'], 1)], '-145016429#0': [(['-145016429#0'], 1)], '-145016429#1': [(['-145016429#1'], 1)], '-145019295#0': [(['-145019295#0'], 1)], '-145019295#1': [(['-145019295#1'], 1)], '-145019295#2': [(['-145019295#2'], 1)], '-145019303': [(['-145019303'], 1)], '-146660067': [(['-146660067'], 1)], '-147175277#0': [(['-147175277#0'], 1)], '-147175277#1': [(['-147175277#1'], 1)], '-163757309#0': [(['-163757309#0'], 1)], '-163757309#1': [(['-163757309#1'], 1)], '-164013612#0': [(['-164013612#0'], 1)], '-164013612#1': [(['-164013612#1'], 1)], '-168553861#0': [(['-168553861#0'], 1)], '-168553861#1': [(['-168553861#1'], 1)], '-168734390': [(['-168734390'], 1)], '-168770476': [(['-168770476'], 1)], '-168773897#0': [(['-168773897#0'], 1)], '-168773897#1': [(['-168773897#1'], 1)], '-168773904#0': [(['-168773904#0'], 1)], '-168773904#1': [(['-168773904#1'], 1)], '-168773904#2': [(['-168773904#2'], 1)], '-168773931': [(['-168773931'], 1)], '-172027185#0': [(['-172027185#0'], 1)], '-172027185#1': [(['-172027185#1'], 1)], '-172027185#2': [(['-172027185#2'], 1)], '-172323220': [(['-172323220'], 1)], '-172449302': [(['-172449302'], 1)], '-172813931': [(['-172813931'], 1)], '-172819166#0': [(['-172819166#0'], 1)], '-172819166#1': [(['-172819166#1'], 1)], '-172829219': [(['-172829219'], 1)], '-173512376#0': [(['-173512376#0'], 1)], '-173512376#1': [(['-173512376#1'], 1)], '-173512377#0': [(['-173512377#0'], 1)], '-173512377#1': [(['-173512377#1'], 1)], '-173512377#2': [(['-173512377#2'], 1)], '-174443898': [(['-174443898'], 1)], '-182246023': [(['-182246023'], 1)], '-182246025#0': [(['-182246025#0'], 1)], '-182246025#1': [(['-182246025#1'], 1)], '-182246025#2': [(['-182246025#2'], 1)], '-182835450': [(['-182835450'], 1)], '-183791752#0': [(['-183791752#0'], 1)], '-183791752#1': [(['-183791752#1'], 1)], '-183873021#0': [(['-183873021#0'], 1)], '-183873021#1': [(['-183873021#1'], 1)], '-184221378': [(['-184221378'], 1)], '-185168107#0': [(['-185168107#0'], 1)], '-185168107#1': [(['-185168107#1'], 1)], '-185816848#0': [(['-185816848#0'], 1)], '-185816848#1': [(['-185816848#1'], 1)], '-193018833': [(['-193018833'], 1)], '-194011454#0': [(['-194011454#0'], 1)], '-194011454#1': [(['-194011454#1'], 1)], '-194011461': [(['-194011461'], 1)], '-194831663': [(['-194831663'], 1)], '-196929000#0': [(['-196929000#0'], 1)], '-196929000#1': [(['-196929000#1'], 1)], '-196929000#2': [(['-196929000#2'], 1)], '-196929008#0': [(['-196929008#0'], 1)], '-196929008#1': [(['-196929008#1'], 1)], '-197253708#0': [(['-197253708#0'], 1)], '-197253708#1': [(['-197253708#1'], 1)], '-197253708#2': [(['-197253708#2'], 1)], '-198966168#0': [(['-198966168#0'], 1)], '-198966168#1': [(['-198966168#1'], 1)], '-198966168#2': [(['-198966168#2'], 1)], '-198966170#0': [(['-198966170#0'], 1)], '-198966170#1': [(['-198966170#1'], 1)], '-23084079': [(['-23084079'], 1)], '-23227351#0': [(['-23227351#0'], 1)], '-23227351#1': [(['-23227351#1'], 1)], '-23227427': [(['-23227427'], 1)], '-234312587': [(['-234312587'], 1)], '-235640906': [(['-235640906'], 1)], '-23729238#0': [(['-23729238#0'], 1)], '-23729238#1': [(['-23729238#1'], 1)], '-23729251#0': [(['-23729251#0'], 1)], '-23729251#1': [(['-23729251#1'], 1)], '-23729251#2': [(['-23729251#2'], 1)], '-24319228#0': [(['-24319228#0'], 1)], '-24319228#1': [(['-24319228#1'], 1)], '-24319228#2': [(['-24319228#2'], 1)], '-24815663': [(['-24815663'], 1)], '-24971165#0': [(['-24971165#0'], 1)], '-24971165#1': [(['-24971165#1'], 1)], '-25096211#0': [(['-25096211#0'], 1)], '-25096211#1': [(['-25096211#1'], 1)], '-252376272#0': [(['-252376272#0'], 1)], '-252376272#1': [(['-252376272#1'], 1)], '-252376272#2': [(['-252376272#2'], 1)], '-254834351#0': [(['-254834351#0'], 1)], '-254834351#1': [(['-254834351#1'], 1)], '-26612692#0': [(['-26612692#0'], 1)], '-26612692#1': [(['-26612692#1'], 1)], '-267227307': [(['-267227307'], 1)], '-271008065#0': [(['-271008065#0'], 1)], '-271008065#1': [(['-271008065#1'], 1)], '-272691807': [(['-272691807'], 1)], '-273277500': [(['-273277500'], 1)], '-273277509#0': [(['-273277509#0'], 1)], '-273277509#1': [(['-273277509#1'], 1)], '-273287556#0': [(['-273287556#0'], 1)], '-273287556#1': [(['-273287556#1'], 1)], '-273287562': [(['-273287562'], 1)], '-273287563': [(['-273287563'], 1)], '-273287567': [(['-273287567'], 1)], '-273287568': [(['-273287568'], 1)], '-273293260': [(['-273293260'], 1)], '-273361477#0': [(['-273361477#0'], 1)], '-273361477#1': [(['-273361477#1'], 1)], '-273362685': [(['-273362685'], 1)], '-273362923#0': [(['-273362923#0'], 1)], '-273362923#1': [(['-273362923#1'], 1)], '-273365286': [(['-273365286'], 1)], '-273365288': [(['-273365288'], 1)], '-273365290': [(['-273365290'], 1)], '-273367398': [(['-273367398'], 1)], '-273367400': [(['-273367400'], 1)], '-273367402#0': [(['-273367402#0'], 1)], '-273367402#1': [(['-273367402#1'], 1)], '-273367405': [(['-273367405'], 1)], '-273367407#0': [(['-273367407#0'], 1)], '-273367407#1': [(['-273367407#1'], 1)], '-273375864': [(['-273375864'], 1)], '-273455653#0': [(['-273455653#0'], 1)], '-273455653#1': [(['-273455653#1'], 1)], '-273455653#2': [(['-273455653#2'], 1)], '-273455654#0': [(['-273455654#0'], 1)], '-273455654#1': [(['-273455654#1'], 1)], '-273455654#2': [(['-273455654#2'], 1)], '-273455654#3': [(['-273455654#3'], 1)], '-273712495': [(['-273712495'], 1)], '-273712496': [(['-273712496'], 1)], '-273714456#0': [(['-273714456#0'], 1)], '-273714456#1': [(['-273714456#1'], 1)], '-273717300#0': [(['-273717300#0'], 1)], '-273717300#1': [(['-273717300#1'], 1)], '-273717306': [(['-273717306'], 1)], '-273741631#0': [(['-273741631#0'], 1)], '-273741631#1': [(['-273741631#1'], 1)], '-273950073': [(['-273950073'], 1)], '-274416153#0': [(['-274416153#0'], 1)], '-274416153#1': [(['-274416153#1'], 1)], '-274825184': [(['-274825184'], 1)], '-274932988': [(['-274932988'], 1)], '-274934659#0': [(['-274934659#0'], 1)], '-274934659#1': [(['-274934659#1'], 1)], '-274938341': [(['-274938341'], 1)], '-274985648#0': [(['-274985648#0'], 1)], '-274985648#1': [(['-274985648#1'], 1)], '-274985649#0': [(['-274985649#0'], 1)], '-274985649#1': [(['-274985649#1'], 1)], '-274985650#0': [(['-274985650#0'], 1)], '-274985650#1': [(['-274985650#1'], 1)], '-274988114#0': [(['-274988114#0'], 1)], '-274988114#1': [(['-274988114#1'], 1)], '-274988114#2': [(['-274988114#2'], 1)], '-274988315#0': [(['-274988315#0'], 1)], '-274988315#1': [(['-274988315#1'], 1)], '-275186710': [(['-275186710'], 1)], '-275258954': [(['-275258954'], 1)], '-275258955': [(['-275258955'], 1)], '-275258956': [(['-275258956'], 1)], '-27755975#0': [(['-27755975#0'], 1)], '-27755975#1': [(['-27755975#1'], 1)], '-27756045': [(['-27756045'], 1)], '-27756067': [(['-27756067'], 1)], '-27756068': [(['-27756068'], 1)], '-277611272#0': [(['-277611272#0'], 1)], '-277611272#1': [(['-277611272#1'], 1)], '-277611272#2': [(['-277611272#2'], 1)], '-277611272#3': [(['-277611272#3'], 1)], '-277611282#0': [(['-277611282#0'], 1)], '-277611282#1': [(['-277611282#1'], 1)], '-277611282#2': [(['-277611282#2'], 1)], '-277611282#3': [(['-277611282#3'], 1)], '-277620581': [(['-277620581'], 1)], '-277620582#0': [(['-277620582#0'], 1)], '-277620582#1': [(['-277620582#1'], 1)], '-277620583#0': [(['-277620583#0'], 1)], '-277620583#1': [(['-277620583#1'], 1)], '-277620584#0': [(['-277620584#0'], 1)], '-277620584#1': [(['-277620584#1'], 1)], '-277620584#2': [(['-277620584#2'], 1)], '-277620585#0': [(['-277620585#0'], 1)], '-277620585#1': [(['-277620585#1'], 1)], '-277620586#0': [(['-277620586#0'], 1)], '-277620586#1': [(['-277620586#1'], 1)], '-277620589#0': [(['-277620589#0'], 1)], '-277620589#1': [(['-277620589#1'], 1)], '-277620589#2': [(['-277620589#2'], 1)], '-277621242': [(['-277621242'], 1)], '-279157858': [(['-279157858'], 1)], '-279157860#0': [(['-279157860#0'], 1)], '-279157860#1': [(['-279157860#1'], 1)], '-279157861': [(['-279157861'], 1)], '-279157862#0': [(['-279157862#0'], 1)], '-279157862#1': [(['-279157862#1'], 1)], '-280774527': [(['-280774527'], 1)], '-280775060': [(['-280775060'], 1)], '-284970400': [(['-284970400'], 1)], '-284970401': [(['-284970401'], 1)], '-285197070#0': [(['-285197070#0'], 1)], '-285197070#1': [(['-285197070#1'], 1)], '-285197078': [(['-285197078'], 1)], '-285197079#0': [(['-285197079#0'], 1)], '-285197079#1': [(['-285197079#1'], 1)], '-285197079#2': [(['-285197079#2'], 1)], '-285197079#3': [(['-285197079#3'], 1)], '-286577466#0': [(['-286577466#0'], 1)], '-286577466#1': [(['-286577466#1'], 1)], '-28660861': [(['-28660861'], 1)], '-288311235': [(['-288311235'], 1)], '-288311236#0': [(['-288311236#0'], 1)], '-288311236#1': [(['-288311236#1'], 1)], '-288313992': [(['-288313992'], 1)], '-288313993#0': [(['-288313993#0'], 1)], '-288313993#1': [(['-288313993#1'], 1)], '-288313993#2': [(['-288313993#2'], 1)], '-288931912#0': [(['-288931912#0'], 1)], '-288931912#1': [(['-288931912#1'], 1)], '-289381224': [(['-289381224'], 1)], '-297323851': [(['-297323851'], 1)], '-298664363': [(['-298664363'], 1)], '-299646741': [(['-299646741'], 1)], '-299735473#0': [(['-299735473#0'], 1)], '-299735473#1': [(['-299735473#1'], 1)], '-300981654': [(['-300981654'], 1)], '-301140399': [(['-301140399'], 1)], '-305863481': [(['-305863481'], 1)], '-307140232': [(['-307140232'], 1)], '-31898400': [(['-31898400'], 1)], '-319913619#0': [(['-319913619#0'], 1)], '-319913619#1': [(['-319913619#1'], 1)], '-324009323#0': [(['-324009323#0'], 1)], '-324009323#1': [(['-324009323#1'], 1)], '-324009323#2': [(['-324009323#2'], 1)], '-328983401#0': [(['-328983401#0'], 1)], '-328983401#1': [(['-328983401#1'], 1)], '-37732652#0': [(['-37732652#0'], 1)], '-37732652#1': [(['-37732652#1'], 1)], '-379871905#0': [(['-379871905#0'], 1)], '-379871905#1': [(['-379871905#1'], 1)], '-379872550': [(['-379872550'], 1)], '-379908019#0': [(['-379908019#0'], 1)], '-379908019#1': [(['-379908019#1'], 1)], '-400378403': [(['-400378403'], 1)], '-400378404': [(['-400378404'], 1)], '-400378406#0': [(['-400378406#0'], 1)], '-400378406#1': [(['-400378406#1'], 1)], '-400608141': [(['-400608141'], 1)], '-42453104#0': [(['-42453104#0'], 1)], '-42453104#1': [(['-42453104#1'], 1)], '-42453343#0': [(['-42453343#0'], 1)], '-42453343#1': [(['-42453343#1'], 1)], '-42453343#2': [(['-42453343#2'], 1)], '-42453343#3': [(['-42453343#3'], 1)], '-42453415': [(['-42453415'], 1)], '-43016041#0': [(['-43016041#0'], 1)], '-43016041#1': [(['-43016041#1'], 1)], '-43061150#0': [(['-43061150#0'], 1)], '-43061150#1': [(['-43061150#1'], 1)], '-43061150#2': [(['-43061150#2'], 1)], '-43061361#0': [(['-43061361#0'], 1)], '-43061361#1': [(['-43061361#1'], 1)], '-43160119#0': [(['-43160119#0'], 1)], '-43160119#1': [(['-43160119#1'], 1)], '-444404990': [(['-444404990'], 1)], '-44935719': [(['-44935719'], 1)], '-4519675': [(['-4519675'], 1)], '-4520048#0': [(['-4520048#0'], 1)], '-4520048#1': [(['-4520048#1'], 1)], '-4520100#0': [(['-4520100#0'], 1)], '-4520100#1': [(['-4520100#1'], 1)], '-464868336#0': [(['-464868336#0'], 1)], '-464868336#1': [(['-464868336#1'], 1)], '-465958707': [(['-465958707'], 1)], '-4868903#0': [(['-4868903#0'], 1)], '-4868903#1': [(['-4868903#1'], 1)], '-4868922#0': [(['-4868922#0'], 1)], '-4868922#1': [(['-4868922#1'], 1)], '-4868922#2': [(['-4868922#2'], 1)], '-4868922#3': [(['-4868922#3'], 1)], '-4868922#4': [(['-4868922#4'], 1)], '-491530383': [(['-491530383'], 1)], '-492314001#0': [(['-492314001#0'], 1)], '-492314001#1': [(['-492314001#1'], 1)], '-492314001#2': [(['-492314001#2'], 1)], '-5110264': [(['-5110264'], 1)], '-515460394': [(['-515460394'], 1)], '-53896956': [(['-53896956'], 1)], '-53896958': [(['-53896958'], 1)], '-54643434': [(['-54643434'], 1)], '-56935336': [(['-56935336'], 1)], '-57742680#0': [(['-57742680#0'], 1)], '-57742680#1': [(['-57742680#1'], 1)], '-58666560#0': [(['-58666560#0'], 1)], '-58666560#1': [(['-58666560#1'], 1)], '-58666560#2': [(['-58666560#2'], 1)], '-58666560#3': [(['-58666560#3'], 1)], '-586824803#0': [(['-586824803#0'], 1)], '-586824803#1': [(['-586824803#1'], 1)], '-598535494': [(['-598535494'], 1)], '-60005026': [(['-60005026'], 1)], '-621957866': [(['-621957866'], 1)], '-79673985': [(['-79673985'], 1)], '-8132447#0': [(['-8132447#0'], 1)], '-8132447#1': [(['-8132447#1'], 1)], '-8311371#0': [(['-8311371#0'], 1)], '-8311371#1': [(['-8311371#1'], 1)], '-8423865': [(['-8423865'], 1)], '-86059925': [(['-86059925'], 1)], '100183072#0': [(['100183072#0'], 1)], '100183072#1': [(['100183072#1'], 1)], '100183073': [(['100183073'], 1)], '100183079#0': [(['100183079#0'], 1)], '100183079#1': [(['100183079#1'], 1)], '100183081#0': [(['100183081#0'], 1)], '100183081#1': [(['100183081#1'], 1)], '100183081#2': [(['100183081#2'], 1)], '100183083#0': [(['100183083#0'], 1)], '100183083#1': [(['100183083#1'], 1)], '10516167#0': [(['10516167#0'], 1)], '10516167#1': [(['10516167#1'], 1)], '10516167#2': [(['10516167#2'], 1)], '10516167#3': [(['10516167#3'], 1)], '10516168#0': [(['10516168#0'], 1)], '10516168#1': [(['10516168#1'], 1)], '10516168#2': [(['10516168#2'], 1)], '10516169#0': [(['10516169#0'], 1)], '10516169#1': [(['10516169#1'], 1)], '10753878#0': [(['10753878#0'], 1)], '10753878#1': [(['10753878#1'], 1)], '111459945#0': [(['111459945#0'], 1)], '111459945#1': [(['111459945#1'], 1)], '111655182#0': [(['111655182#0'], 1)], '111655182#1': [(['111655182#1'], 1)], '111737333': [(['111737333'], 1)], '111737334': [(['111737334'], 1)], '111737367#0': [(['111737367#0'], 1)], '111737367#1': [(['111737367#1'], 1)], '111737367#2': [(['111737367#2'], 1)], '111866281#0': [(['111866281#0'], 1)], '111866281#1': [(['111866281#1'], 1)], '111866281#2': [(['111866281#2'], 1)], '112602522': [(['112602522'], 1)], '112602524#0': [(['112602524#0'], 1)], '112602524#1': [(['112602524#1'], 1)], '112602524#2': [(['112602524#2'], 1)], '112602525': [(['112602525'], 1)], '114187491#0': [(['114187491#0'], 1)], '114187491#1': [(['114187491#1'], 1)], '114187491#2': [(['114187491#2'], 1)], '116883176': [(['116883176'], 1)], '120458050': [(['120458050'], 1)], '120458052': [(['120458052'], 1)], '120458053#0': [(['120458053#0'], 1)], '120458053#1': [(['120458053#1'], 1)], '120458054#0': [(['120458054#0'], 1)], '120458054#1': [(['120458054#1'], 1)], '120458054#2': [(['120458054#2'], 1)], '120458056': [(['120458056'], 1)], '12545558#0': [(['12545558#0'], 1)], '12545558#1': [(['12545558#1'], 1)], '134235552#0': [(['134235552#0'], 1)], '134235552#1': [(['134235552#1'], 1)], '134235719#0': [(['134235719#0'], 1)], '134235719#1': [(['134235719#1'], 1)], '134397031#0': [(['134397031#0'], 1)], '134397031#1': [(['134397031#1'], 1)], '134397031#2': [(['134397031#2'], 1)], '136990352#0': [(['136990352#0'], 1)], '136990352#1': [(['136990352#1'], 1)], '138141246#0': [(['138141246#0'], 1)], '138141246#1': [(['138141246#1'], 1)], '138141249#0': [(['138141249#0'], 1)], '138141249#1': [(['138141249#1'], 1)], '138141249#2': [(['138141249#2'], 1)], '138141254#0': [(['138141254#0'], 1)], '138141254#1': [(['138141254#1'], 1)], '138141254#2': [(['138141254#2'], 1)], '139612784': [(['139612784'], 1)], '141633903': [(['141633903'], 1)], '141633904#0': [(['141633904#0'], 1)], '141633904#1': [(['141633904#1'], 1)], '141633904#2': [(['141633904#2'], 1)], '141633904#3': [(['141633904#3'], 1)], '141633904#4': [(['141633904#4'], 1)], '141633904#5': [(['141633904#5'], 1)], '141633904#6': [(['141633904#6'], 1)], '141633904#7': [(['141633904#7'], 1)], '141634886': [(['141634886'], 1)], '142387174': [(['142387174'], 1)], '142387176#0': [(['142387176#0'], 1)], '142387176#1': [(['142387176#1'], 1)], '143276335': [(['143276335'], 1)], '143276337': [(['143276337'], 1)], '14348297': [(['14348297'], 1)], '145014439': [(['145014439'], 1)], '145014444#0': [(['145014444#0'], 1)], '145014444#1': [(['145014444#1'], 1)], '145014444#2': [(['145014444#2'], 1)], '145016429#0': [(['145016429#0'], 1)], '145016429#1': [(['145016429#1'], 1)], '145018315#0': [(['145018315#0'], 1)], '145018315#1': [(['145018315#1'], 1)], '145018316': [(['145018316'], 1)], '145018317': [(['145018317'], 1)], '145019295#0': [(['145019295#0'], 1)], '145019295#1': [(['145019295#1'], 1)], '145019295#2': [(['145019295#2'], 1)], '145019298#0': [(['145019298#0'], 1)], '145019298#1': [(['145019298#1'], 1)], '145019298#2': [(['145019298#2'], 1)], '145019300#0': [(['145019300#0'], 1)], '145019300#1': [(['145019300#1'], 1)], '145019303': [(['145019303'], 1)], '146660067': [(['146660067'], 1)], '147175265#0': [(['147175265#0'], 1)], '147175265#1': [(['147175265#1'], 1)], '147175268': [(['147175268'], 1)], '147175277#0': [(['147175277#0'], 1)], '147175277#1': [(['147175277#1'], 1)], '161068026#0': [(['161068026#0'], 1)], '161068026#1': [(['161068026#1'], 1)], '163757282#0': [(['163757282#0'], 1)], '163757282#1': [(['163757282#1'], 1)], '163757284#0': [(['163757284#0'], 1)], '163757284#1': [(['163757284#1'], 1)], '163757284#2': [(['163757284#2'], 1)], '163757290': [(['163757290'], 1)], '163757294#0': [(['163757294#0'], 1)], '163757294#1': [(['163757294#1'], 1)], '163757297#0': [(['163757297#0'], 1)], '163757297#1': [(['163757297#1'], 1)], '163757297#2': [(['163757297#2'], 1)], '163757309#0': [(['163757309#0'], 1)], '163757309#1': [(['163757309#1'], 1)], '164013612#0': [(['164013612#0'], 1)], '164013612#1': [(['164013612#1'], 1)], '164013613#0': [(['164013613#0'], 1)], '164013613#1': [(['164013613#1'], 1)], '164013613#2': [(['164013613#2'], 1)], '164070441': [(['164070441'], 1)], '164070445': [(['164070445'], 1)], '164070447': [(['164070447'], 1)], '164070451#0': [(['164070451#0'], 1)], '164070451#1': [(['164070451#1'], 1)], '164070453': [(['164070453'], 1)], '164070455': [(['164070455'], 1)], '164111015#0': [(['164111015#0'], 1)], '164111015#1': [(['164111015#1'], 1)], '164111015#2': [(['164111015#2'], 1)], '164111025': [(['164111025'], 1)], '165090432#0': [(['165090432#0'], 1)], '165090432#1': [(['165090432#1'], 1)], '165090432#2': [(['165090432#2'], 1)], '165090435#0': [(['165090435#0'], 1)], '165090435#1': [(['165090435#1'], 1)], '165847251#0': [(['165847251#0'], 1)], '165847251#1': [(['165847251#1'], 1)], '165847251#2': [(['165847251#2'], 1)], '165990904#0': [(['165990904#0'], 1)], '165990904#1': [(['165990904#1'], 1)], '165990904#2': [(['165990904#2'], 1)], '165990904#3': [(['165990904#3'], 1)], '165990905': [(['165990905'], 1)], '165990906#0': [(['165990906#0'], 1)], '165990906#1': [(['165990906#1'], 1)], '165990906#2': [(['165990906#2'], 1)], '165990907#0': [(['165990907#0'], 1)], '165990907#1': [(['165990907#1'], 1)], '165990908#0': [(['165990908#0'], 1)], '165990908#1': [(['165990908#1'], 1)], '165990910#0': [(['165990910#0'], 1)], '165990910#1': [(['165990910#1'], 1)], '165990910#2': [(['165990910#2'], 1)], '166039358': [(['166039358'], 1)], '166039362#0': [(['166039362#0'], 1)], '166039362#1': [(['166039362#1'], 1)], '166039363': [(['166039363'], 1)], '166623697#0': [(['166623697#0'], 1)], '166623697#1': [(['166623697#1'], 1)], '166623697#2': [(['166623697#2'], 1)], '166623699#0': [(['166623699#0'], 1)], '166623699#1': [(['166623699#1'], 1)], '168553840': [(['168553840'], 1)], '168553854#0': [(['168553854#0'], 1)], '168553854#1': [(['168553854#1'], 1)], '168553855#0': [(['168553855#0'], 1)], '168553855#1': [(['168553855#1'], 1)], '168553855#2': [(['168553855#2'], 1)], '168553861#0': [(['168553861#0'], 1)], '168553861#1': [(['168553861#1'], 1)], '168553873#0': [(['168553873#0'], 1)], '168553873#1': [(['168553873#1'], 1)], '168553878': [(['168553878'], 1)], '168553882': [(['168553882'], 1)], '168553883#0': [(['168553883#0'], 1)], '168553883#1': [(['168553883#1'], 1)], '168553885#0': [(['168553885#0'], 1)], '168553885#1': [(['168553885#1'], 1)], '168553885#2': [(['168553885#2'], 1)], '168553885#3': [(['168553885#3'], 1)], '168553886': [(['168553886'], 1)], '168553887': [(['168553887'], 1)], '168553888': [(['168553888'], 1)], '168734353#0': [(['168734353#0'], 1)], '168734353#1': [(['168734353#1'], 1)], '168734374': [(['168734374'], 1)], '168734390': [(['168734390'], 1)], '168770476': [(['168770476'], 1)], '168770482#0': [(['168770482#0'], 1)], '168770482#1': [(['168770482#1'], 1)], '168770482#2': [(['168770482#2'], 1)], '168770493': [(['168770493'], 1)], '168770497#0': [(['168770497#0'], 1)], '168770497#1': [(['168770497#1'], 1)], '168770526#0': [(['168770526#0'], 1)], '168770526#1': [(['168770526#1'], 1)], '168770528': [(['168770528'], 1)], '168770529': [(['168770529'], 1)], '168770553': [(['168770553'], 1)], '168773897#0': [(['168773897#0'], 1)], '168773897#1': [(['168773897#1'], 1)], '168773901': [(['168773901'], 1)], '168773904#0': [(['168773904#0'], 1)], '168773904#1': [(['168773904#1'], 1)], '168773904#2': [(['168773904#2'], 1)], '168773918': [(['168773918'], 1)], '168773931': [(['168773931'], 1)], '168773934': [(['168773934'], 1)], '171931724#0': [(['171931724#0'], 1)], '171931724#1': [(['171931724#1'], 1)], '171941225': [(['171941225'], 1)], '171941226#0': [(['171941226#0'], 1)], '171941226#1': [(['171941226#1'], 1)], '171941230': [(['171941230'], 1)], '172027185#0': [(['172027185#0'], 1)], '172027185#1': [(['172027185#1'], 1)], '172027185#2': [(['172027185#2'], 1)], '172323219': [(['172323219'], 1)], '172323220': [(['172323220'], 1)], '172323221': [(['172323221'], 1)], '172326974': [(['172326974'], 1)], '172345033#0': [(['172345033#0'], 1)], '172345033#1': [(['172345033#1'], 1)], '172449302': [(['172449302'], 1)], '172813930#0': [(['172813930#0'], 1)], '172813930#1': [(['172813930#1'], 1)], '172813931': [(['172813931'], 1)], '172819166#0': [(['172819166#0'], 1)], '172819166#1': [(['172819166#1'], 1)], '172819167#0': [(['172819167#0'], 1)], '172819167#1': [(['172819167#1'], 1)], '172824078#0': [(['172824078#0'], 1)], '172824078#1': [(['172824078#1'], 1)], '172826555#0': [(['172826555#0'], 1)], '172826555#1': [(['172826555#1'], 1)], '172826555#2': [(['172826555#2'], 1)], '172829219': [(['172829219'], 1)], '173512368#0': [(['173512368#0'], 1)], '173512368#1': [(['173512368#1'], 1)], '173512368#2': [(['173512368#2'], 1)], '173512371#0': [(['173512371#0'], 1)], '173512371#1': [(['173512371#1'], 1)], '173512371#2': [(['173512371#2'], 1)], '173512376#0': [(['173512376#0'], 1)], '173512376#1': [(['173512376#1'], 1)], '173512377#0': [(['173512377#0'], 1)], '173512377#1': [(['173512377#1'], 1)], '173512377#2': [(['173512377#2'], 1)], '174443896#0': [(['174443896#0'], 1)], '174443896#1': [(['174443896#1'], 1)], '174443896#2': [(['174443896#2'], 1)], '174443897#0': [(['174443897#0'], 1)], '174443897#1': [(['174443897#1'], 1)], '174443897#2': [(['174443897#2'], 1)], '174443897#3': [(['174443897#3'], 1)], '174443898': [(['174443898'], 1)], '174463163#0': [(['174463163#0'], 1)], '174463163#1': [(['174463163#1'], 1)], '174463166#0': [(['174463166#0'], 1)], '174463166#1': [(['174463166#1'], 1)], '174463167#0': [(['174463167#0'], 1)], '174463167#1': [(['174463167#1'], 1)], '174463167#2': [(['174463167#2'], 1)], '174463167#3': [(['174463167#3'], 1)], '174463167#4': [(['174463167#4'], 1)], '174923206#0': [(['174923206#0'], 1)], '174923206#1': [(['174923206#1'], 1)], '182196403': [(['182196403'], 1)], '182246023': [(['182246023'], 1)], '182246025#0': [(['182246025#0'], 1)], '182246025#1': [(['182246025#1'], 1)], '182246025#2': [(['182246025#2'], 1)], '182835450': [(['182835450'], 1)], '183444436': [(['183444436'], 1)], '183444438': [(['183444438'], 1)], '183791752#0': [(['183791752#0'], 1)], '183791752#1': [(['183791752#1'], 1)], '183873021#0': [(['183873021#0'], 1)], '183873021#1': [(['183873021#1'], 1)], '183873025#0': [(['183873025#0'], 1)], '183873025#1': [(['183873025#1'], 1)], '183873025#2': [(['183873025#2'], 1)], '183873025#3': [(['183873025#3'], 1)], '184221378': [(['184221378'], 1)], '185168099': [(['185168099'], 1)], '185168107#0': [(['185168107#0'], 1)], '185168107#1': [(['185168107#1'], 1)], '185168110#0': [(['185168110#0'], 1)], '185168110#1': [(['185168110#1'], 1)], '185310341#0': [(['185310341#0'], 1)], '185310341#1': [(['185310341#1'], 1)], '185310341#2': [(['185310341#2'], 1)], '185310341#3': [(['185310341#3'], 1)], '185310346#0': [(['185310346#0'], 1)], '185310346#1': [(['185310346#1'], 1)], '185310346#2': [(['185310346#2'], 1)], '185310346#3': [(['185310346#3'], 1)], '185816848#0': [(['185816848#0'], 1)], '185816848#1': [(['185816848#1'], 1)], '186691901#0': [(['186691901#0'], 1)], '186691901#1': [(['186691901#1'], 1)], '186691901#2': [(['186691901#2'], 1)], '186691901#3': [(['186691901#3'], 1)], '186691901#4': [(['186691901#4'], 1)], '189272428#0': [(['189272428#0'], 1)], '189272428#1': [(['189272428#1'], 1)], '193018833': [(['193018833'], 1)], '194011454#0': [(['194011454#0'], 1)], '194011454#1': [(['194011454#1'], 1)], '194011456#0': [(['194011456#0'], 1)], '194011456#1': [(['194011456#1'], 1)], '194011461': [(['194011461'], 1)], '194011463': [(['194011463'], 1)], '194011466': [(['194011466'], 1)], '194831663': [(['194831663'], 1)], '194834342#0': [(['194834342#0'], 1)], '194834342#1': [(['194834342#1'], 1)], '196012018#0': [(['196012018#0'], 1)], '196012018#1': [(['196012018#1'], 1)], '196928982#0': [(['196928982#0'], 1)], '196928982#1': [(['196928982#1'], 1)], '196928983#0': [(['196928983#0'], 1)], '196928983#1': [(['196928983#1'], 1)], '196928988': [(['196928988'], 1)], '196928993#0': [(['196928993#0'], 1)], '196928993#1': [(['196928993#1'], 1)], '196929000#0': [(['196929000#0'], 1)], '196929000#1': [(['196929000#1'], 1)], '196929000#2': [(['196929000#2'], 1)], '196929008#0': [(['196929008#0'], 1)], '196929008#1': [(['196929008#1'], 1)], '197249314': [(['197249314'], 1)], '197249315': [(['197249315'], 1)], '197253708#0': [(['197253708#0'], 1)], '197253708#1': [(['197253708#1'], 1)], '197253708#2': [(['197253708#2'], 1)], '198966168#0': [(['198966168#0'], 1)], '198966168#1': [(['198966168#1'], 1)], '198966168#2': [(['198966168#2'], 1)], '198966170#0': [(['198966170#0'], 1)], '198966170#1': [(['198966170#1'], 1)], '200401770#0': [(['200401770#0'], 1)], '200401770#1': [(['200401770#1'], 1)], '200401771#0': [(['200401771#0'], 1)], '200401771#1': [(['200401771#1'], 1)], '222858930#0': [(['222858930#0'], 1)], '222858930#1': [(['222858930#1'], 1)], '222858930#2': [(['222858930#2'], 1)], '222858930#3': [(['222858930#3'], 1)], '226139959': [(['226139959'], 1)], '226139960': [(['226139960'], 1)], '230638667': [(['230638667'], 1)], '23084025': [(['23084025'], 1)], '23084079': [(['23084079'], 1)], '23084109': [(['23084109'], 1)], '23227334#0': [(['23227334#0'], 1)], '23227334#1': [(['23227334#1'], 1)], '23227347#0': [(['23227347#0'], 1)], '23227347#1': [(['23227347#1'], 1)], '23227347#2': [(['23227347#2'], 1)], '23227347#3': [(['23227347#3'], 1)], '23227351#0': [(['23227351#0'], 1)], '23227351#1': [(['23227351#1'], 1)], '23227359#0': [(['23227359#0'], 1)], '23227359#1': [(['23227359#1'], 1)], '23227359#2': [(['23227359#2'], 1)], '23227361#0': [(['23227361#0'], 1)], '23227361#1': [(['23227361#1'], 1)], '23227363#0': [(['23227363#0'], 1)], '23227363#1': [(['23227363#1'], 1)], '23227363#2': [(['23227363#2'], 1)], '23227363#3': [(['23227363#3'], 1)], '23227364': [(['23227364'], 1)], '23227375#0': [(['23227375#0'], 1)], '23227375#1': [(['23227375#1'], 1)], '23227377#0': [(['23227377#0'], 1)], '23227377#1': [(['23227377#1'], 1)], '23227377#2': [(['23227377#2'], 1)], '23227377#3': [(['23227377#3'], 1)], '23227390#0': [(['23227390#0'], 1)], '23227390#1': [(['23227390#1'], 1)], '23227390#2': [(['23227390#2'], 1)], '23227390#3': [(['23227390#3'], 1)], '23227394#0': [(['23227394#0'], 1)], '23227394#1': [(['23227394#1'], 1)], '23227395#0': [(['23227395#0'], 1)], '23227395#1': [(['23227395#1'], 1)], '23227412#0': [(['23227412#0'], 1)], '23227412#1': [(['23227412#1'], 1)], '23227412#2': [(['23227412#2'], 1)], '23227412#3': [(['23227412#3'], 1)], '23227421#0': [(['23227421#0'], 1)], '23227421#1': [(['23227421#1'], 1)], '23227421#2': [(['23227421#2'], 1)], '23227421#3': [(['23227421#3'], 1)], '23227421#4': [(['23227421#4'], 1)], '23227426': [(['23227426'], 1)], '23227427': [(['23227427'], 1)], '23227429#0': [(['23227429#0'], 1)], '23227429#1': [(['23227429#1'], 1)], '23227429#2': [(['23227429#2'], 1)], '23227471#0': [(['23227471#0'], 1)], '23227471#1': [(['23227471#1'], 1)], '23227471#2': [(['23227471#2'], 1)], '23227528#0': [(['23227528#0'], 1)], '23227528#1': [(['23227528#1'], 1)], '23227528#2': [(['23227528#2'], 1)], '23227539': [(['23227539'], 1)], '23227549#0': [(['23227549#0'], 1)], '23227549#1': [(['23227549#1'], 1)], '233634300#0': [(['233634300#0'], 1)], '233634300#1': [(['233634300#1'], 1)], '233634301': [(['233634301'], 1)], '234305177#0': [(['234305177#0'], 1)], '234305177#1': [(['234305177#1'], 1)], '234305180': [(['234305180'], 1)], '234305185#0': [(['234305185#0'], 1)], '234305185#1': [(['234305185#1'], 1)], '234312587': [(['234312587'], 1)], '235640906': [(['235640906'], 1)], '23729238#0': [(['23729238#0'], 1)], '23729238#1': [(['23729238#1'], 1)], '23729246#0': [(['23729246#0'], 1)], '23729246#1': [(['23729246#1'], 1)], '23729246#2': [(['23729246#2'], 1)], '23729246#3': [(['23729246#3'], 1)], '23729251#0': [(['23729251#0'], 1)], '23729251#1': [(['23729251#1'], 1)], '23729251#2': [(['23729251#2'], 1)], '24319228#0': [(['24319228#0'], 1)], '24319228#1': [(['24319228#1'], 1)], '24319228#2': [(['24319228#2'], 1)], '24606606#0': [(['24606606#0'], 1)], '24606606#1': [(['24606606#1'], 1)], '24606659#0': [(['24606659#0'], 1)], '24606659#1': [(['24606659#1'], 1)], '24606659#2': [(['24606659#2'], 1)], '24606659#3': [(['24606659#3'], 1)], '24606659#4': [(['24606659#4'], 1)], '24711534': [(['24711534'], 1)], '24815663': [(['24815663'], 1)], '24815726#0': [(['24815726#0'], 1)], '24815726#1': [(['24815726#1'], 1)], '24815726#2': [(['24815726#2'], 1)], '24971165#0': [(['24971165#0'], 1)], '24971165#1': [(['24971165#1'], 1)], '25096206#0': [(['25096206#0'], 1)], '25096206#1': [(['25096206#1'], 1)], '25096206#2': [(['25096206#2'], 1)], '25096211#0': [(['25096211#0'], 1)], '25096211#1': [(['25096211#1'], 1)], '25096222#0': [(['25096222#0'], 1)], '25096222#1': [(['25096222#1'], 1)], '251231855': [(['251231855'], 1)], '252376272#0': [(['252376272#0'], 1)], '252376272#1': [(['252376272#1'], 1)], '252376272#2': [(['252376272#2'], 1)], '252415761': [(['252415761'], 1)], '252421997': [(['252421997'], 1)], '252422219#0': [(['252422219#0'], 1)], '252422219#1': [(['252422219#1'], 1)], '252422220#0': [(['252422220#0'], 1)], '252422220#1': [(['252422220#1'], 1)], '252422220#2': [(['252422220#2'], 1)], '252422997': [(['252422997'], 1)], '254834351#0': [(['254834351#0'], 1)], '254834351#1': [(['254834351#1'], 1)], '255054966#0': [(['255054966#0'], 1)], '255054966#1': [(['255054966#1'], 1)], '258514920': [(['258514920'], 1)], '260437782': [(['260437782'], 1)], '26612692#0': [(['26612692#0'], 1)], '26612692#1': [(['26612692#1'], 1)], '26613344#0': [(['26613344#0'], 1)], '26613344#1': [(['26613344#1'], 1)], '26702701#0': [(['26702701#0'], 1)], '26702701#1': [(['26702701#1'], 1)], '267227307': [(['267227307'], 1)], '267227308': [(['267227308'], 1)], '267227309#0': [(['267227309#0'], 1)], '267227309#1': [(['267227309#1'], 1)], '267227310#0': [(['267227310#0'], 1)], '267227310#1': [(['267227310#1'], 1)], '26743565#0': [(['26743565#0'], 1)], '26743565#1': [(['26743565#1'], 1)], '269518392#0': [(['269518392#0'], 1)], '269518392#1': [(['269518392#1'], 1)], '271008065#0': [(['271008065#0'], 1)], '271008065#1': [(['271008065#1'], 1)], '271008067': [(['271008067'], 1)], '271074928': [(['271074928'], 1)], '271079954': [(['271079954'], 1)], '272691807': [(['272691807'], 1)], '273277500': [(['273277500'], 1)], '273277509#0': [(['273277509#0'], 1)], '273277509#1': [(['273277509#1'], 1)], '273287551#0': [(['273287551#0'], 1)], '273287551#1': [(['273287551#1'], 1)], '273287553': [(['273287553'], 1)], '273287554': [(['273287554'], 1)], '273287555': [(['273287555'], 1)], '273287556#0': [(['273287556#0'], 1)], '273287556#1': [(['273287556#1'], 1)], '273287557#0': [(['273287557#0'], 1)], '273287557#1': [(['273287557#1'], 1)], '273287557#2': [(['273287557#2'], 1)], '273287558': [(['273287558'], 1)], '273287559#0': [(['273287559#0'], 1)], '273287559#1': [(['273287559#1'], 1)], '273287560#0': [(['273287560#0'], 1)], '273287560#1': [(['273287560#1'], 1)], '273287561#0': [(['273287561#0'], 1)], '273287561#1': [(['273287561#1'], 1)], '273287562': [(['273287562'], 1)], '273287563': [(['273287563'], 1)], '273287565': [(['273287565'], 1)], '273287566#0': [(['273287566#0'], 1)], '273287566#1': [(['273287566#1'], 1)], '273287567': [(['273287567'], 1)], '273287568': [(['273287568'], 1)], '273293256': [(['273293256'], 1)], '273293257#0': [(['273293257#0'], 1)], '273293257#1': [(['273293257#1'], 1)], '273293257#2': [(['273293257#2'], 1)], '273293259#0': [(['273293259#0'], 1)], '273293259#1': [(['273293259#1'], 1)], '273293259#2': [(['273293259#2'], 1)], '273293260': [(['273293260'], 1)], '273361386#0': [(['273361386#0'], 1)], '273361386#1': [(['273361386#1'], 1)], '273361386#2': [(['273361386#2'], 1)], '273361386#3': [(['273361386#3'], 1)], '273361477#0': [(['273361477#0'], 1)], '273361477#1': [(['273361477#1'], 1)], '273361729#0': [(['273361729#0'], 1)], '273361729#1': [(['273361729#1'], 1)], '273362685': [(['273362685'], 1)], '273362687#0': [(['273362687#0'], 1)], '273362687#1': [(['273362687#1'], 1)], '273362688': [(['273362688'], 1)], '273362923#0': [(['273362923#0'], 1)], '273362923#1': [(['273362923#1'], 1)], '273365286': [(['273365286'], 1)], '273365288': [(['273365288'], 1)], '273365290': [(['273365290'], 1)], '273367398': [(['273367398'], 1)], '273367400': [(['273367400'], 1)], '273367402#0': [(['273367402#0'], 1)], '273367402#1': [(['273367402#1'], 1)], '273367405': [(['273367405'], 1)], '273367407#0': [(['273367407#0'], 1)], '273367407#1': [(['273367407#1'], 1)], '273368819': [(['273368819'], 1)], '273368826': [(['273368826'], 1)], '273368829': [(['273368829'], 1)], '273368832#0': [(['273368832#0'], 1)], '273368832#1': [(['273368832#1'], 1)], '273368832#2': [(['273368832#2'], 1)], '273368837#0': [(['273368837#0'], 1)], '273368837#1': [(['273368837#1'], 1)], '273368839': [(['273368839'], 1)], '273375864': [(['273375864'], 1)], '273375865': [(['273375865'], 1)], '273455646#0': [(['273455646#0'], 1)], '273455646#1': [(['273455646#1'], 1)], '273455647#0': [(['273455647#0'], 1)], '273455647#1': [(['273455647#1'], 1)], '273455647#2': [(['273455647#2'], 1)], '273455647#3': [(['273455647#3'], 1)], '273455647#4': [(['273455647#4'], 1)], '273455648': [(['273455648'], 1)], '273455650#0': [(['273455650#0'], 1)], '273455650#1': [(['273455650#1'], 1)], '273455651': [(['273455651'], 1)], '273455652#0': [(['273455652#0'], 1)], '273455652#1': [(['273455652#1'], 1)], '273455653#0': [(['273455653#0'], 1)], '273455653#1': [(['273455653#1'], 1)], '273455653#2': [(['273455653#2'], 1)], '273455654#0': [(['273455654#0'], 1)], '273455654#1': [(['273455654#1'], 1)], '273455654#2': [(['273455654#2'], 1)], '273455654#3': [(['273455654#3'], 1)], '273455655': [(['273455655'], 1)], '273712491#0': [(['273712491#0'], 1)], '273712491#1': [(['273712491#1'], 1)], '273712493#0': [(['273712493#0'], 1)], '273712493#1': [(['273712493#1'], 1)], '273712493#2': [(['273712493#2'], 1)], '273712493#3': [(['273712493#3'], 1)], '273712494#0': [(['273712494#0'], 1)], '273712494#1': [(['273712494#1'], 1)], '273712495': [(['273712495'], 1)], '273712496': [(['273712496'], 1)], '273714456#0': [(['273714456#0'], 1)], '273714456#1': [(['273714456#1'], 1)], '273714459': [(['273714459'], 1)], '273714460#0': [(['273714460#0'], 1)], '273714460#1': [(['273714460#1'], 1)], '273714460#2': [(['273714460#2'], 1)], '273715467#0': [(['273715467#0'], 1)], '273715467#1': [(['273715467#1'], 1)], '273715467#2': [(['273715467#2'], 1)], '273717300#0': [(['273717300#0'], 1)], '273717300#1': [(['273717300#1'], 1)], '273717301#0': [(['273717301#0'], 1)], '273717301#1': [(['273717301#1'], 1)], '273717302': [(['273717302'], 1)], '273717304': [(['273717304'], 1)], '273717305#0': [(['273717305#0'], 1)], '273717305#1': [(['273717305#1'], 1)], '273717306': [(['273717306'], 1)], '273717307#0': [(['273717307#0'], 1)], '273717307#1': [(['273717307#1'], 1)], '273717308#0': [(['273717308#0'], 1)], '273717308#1': [(['273717308#1'], 1)], '273717309#0': [(['273717309#0'], 1)], '273717309#1': [(['273717309#1'], 1)], '273717500': [(['273717500'], 1)], '273717562#0': [(['273717562#0'], 1)], '273717562#1': [(['273717562#1'], 1)], '273741631#0': [(['273741631#0'], 1)], '273741631#1': [(['273741631#1'], 1)], '273756537': [(['273756537'], 1)], '273950073': [(['273950073'], 1)], '273950074#0': [(['273950074#0'], 1)], '273950074#1': [(['273950074#1'], 1)], '273950075': [(['273950075'], 1)], '273950076': [(['273950076'], 1)], '273954320#0': [(['273954320#0'], 1)], '273954320#1': [(['273954320#1'], 1)], '273954321#0': [(['273954321#0'], 1)], '273954321#1': [(['273954321#1'], 1)], '273959703': [(['273959703'], 1)], '274103569': [(['274103569'], 1)], '274103570': [(['274103570'], 1)], '274103571#0': [(['274103571#0'], 1)], '274103571#1': [(['274103571#1'], 1)], '274103572#0': [(['274103572#0'], 1)], '274103572#1': [(['274103572#1'], 1)], '274103573#0': [(['274103573#0'], 1)], '274103573#1': [(['274103573#1'], 1)], '274103573#2': [(['274103573#2'], 1)], '274103574': [(['274103574'], 1)], '274103575': [(['274103575'], 1)], '274120768#0': [(['274120768#0'], 1)], '274120768#1': [(['274120768#1'], 1)], '274120768#2': [(['274120768#2'], 1)], '274408453': [(['274408453'], 1)], '274408454': [(['274408454'], 1)], '274416153#0': [(['274416153#0'], 1)], '274416153#1': [(['274416153#1'], 1)], '274498111#0': [(['274498111#0'], 1)], '274498111#1': [(['274498111#1'], 1)], '274787683': [(['274787683'], 1)], '274791805': [(['274791805'], 1)], '274791810#0': [(['274791810#0'], 1)], '274791810#1': [(['274791810#1'], 1)], '274791811': [(['274791811'], 1)], '274825180': [(['274825180'], 1)], '274825181#0': [(['274825181#0'], 1)], '274825181#1': [(['274825181#1'], 1)], '274825181#2': [(['274825181#2'], 1)], '274825184': [(['274825184'], 1)], '274825185#0': [(['274825185#0'], 1)], '274825185#1': [(['274825185#1'], 1)], '274825185#2': [(['274825185#2'], 1)], '274827996': [(['274827996'], 1)], '274827998#0': [(['274827998#0'], 1)], '274827998#1': [(['274827998#1'], 1)], '274932988': [(['274932988'], 1)], '274934659#0': [(['274934659#0'], 1)], '274934659#1': [(['274934659#1'], 1)], '274938341': [(['274938341'], 1)], '274938342#0': [(['274938342#0'], 1)], '274938342#1': [(['274938342#1'], 1)], '274938343#0': [(['274938343#0'], 1)], '274938343#1': [(['274938343#1'], 1)], '274984846#0': [(['274984846#0'], 1)], '274984846#1': [(['274984846#1'], 1)], '274984847': [(['274984847'], 1)], '274984848': [(['274984848'], 1)], '274984849': [(['274984849'], 1)], '274984850#0': [(['274984850#0'], 1)], '274984850#1': [(['274984850#1'], 1)], '274984850#2': [(['274984850#2'], 1)], '274984851#0': [(['274984851#0'], 1)], '274984851#1': [(['274984851#1'], 1)], '274984851#2': [(['274984851#2'], 1)], '274984852': [(['274984852'], 1)], '274984853#0': [(['274984853#0'], 1)], '274984853#1': [(['274984853#1'], 1)], '274985326': [(['274985326'], 1)], '274985648#0': [(['274985648#0'], 1)], '274985648#1': [(['274985648#1'], 1)], '274985649#0': [(['274985649#0'], 1)], '274985649#1': [(['274985649#1'], 1)], '274985650#0': [(['274985650#0'], 1)], '274985650#1': [(['274985650#1'], 1)], '274985651#0': [(['274985651#0'], 1)], '274985651#1': [(['274985651#1'], 1)], '274985651#2': [(['274985651#2'], 1)], '274986450#0': [(['274986450#0'], 1)], '274986450#1': [(['274986450#1'], 1)], '274986451': [(['274986451'], 1)], '274987589': [(['274987589'], 1)], '274987591': [(['274987591'], 1)], '274988114#0': [(['274988114#0'], 1)], '274988114#1': [(['274988114#1'], 1)], '274988114#2': [(['274988114#2'], 1)], '274988115#0': [(['274988115#0'], 1)], '274988115#1': [(['274988115#1'], 1)], '274988115#2': [(['274988115#2'], 1)], '274988115#3': [(['274988115#3'], 1)], '274988313': [(['274988313'], 1)], '274988314': [(['274988314'], 1)], '274988315#0': [(['274988315#0'], 1)], '274988315#1': [(['274988315#1'], 1)], '274988316#0': [(['274988316#0'], 1)], '274988316#1': [(['274988316#1'], 1)], '274988316#2': [(['274988316#2'], 1)], '274988316#3': [(['274988316#3'], 1)], '274988410#0': [(['274988410#0'], 1)], '274988410#1': [(['274988410#1'], 1)], '274988411': [(['274988411'], 1)], '275186708#0': [(['275186708#0'], 1)], '275186708#1': [(['275186708#1'], 1)], '275186709': [(['275186709'], 1)], '275186710': [(['275186710'], 1)], '275186712#0': [(['275186712#0'], 1)], '275186712#1': [(['275186712#1'], 1)], '275186713': [(['275186713'], 1)], '275186714': [(['275186714'], 1)], '275186715': [(['275186715'], 1)], '275186717#0': [(['275186717#0'], 1)], '275186717#1': [(['275186717#1'], 1)], '275186718#0': [(['275186718#0'], 1)], '275186718#1': [(['275186718#1'], 1)], '275186719': [(['275186719'], 1)], '275186721#0': [(['275186721#0'], 1)], '275186721#1': [(['275186721#1'], 1)], '275186722': [(['275186722'], 1)], '275186723#0': [(['275186723#0'], 1)], '275186723#1': [(['275186723#1'], 1)], '275186724#0': [(['275186724#0'], 1)], '275186724#1': [(['275186724#1'], 1)], '275186725': [(['275186725'], 1)], '275189750#0': [(['275189750#0'], 1)], '275189750#1': [(['275189750#1'], 1)], '275189750#2': [(['275189750#2'], 1)], '275198424#0': [(['275198424#0'], 1)], '275198424#1': [(['275198424#1'], 1)], '275198427': [(['275198427'], 1)], '275253902': [(['275253902'], 1)], '275253903': [(['275253903'], 1)], '275253905': [(['275253905'], 1)], '275258954': [(['275258954'], 1)], '275258955': [(['275258955'], 1)], '275258956': [(['275258956'], 1)], '275260146#0': [(['275260146#0'], 1)], '275260146#1': [(['275260146#1'], 1)], '276506963': [(['276506963'], 1)], '276507213#0': [(['276507213#0'], 1)], '276507213#1': [(['276507213#1'], 1)], '276507214': [(['276507214'], 1)], '27745094': [(['27745094'], 1)], '27745095#0': [(['27745095#0'], 1)], '27745095#1': [(['27745095#1'], 1)], '27745201#0': [(['27745201#0'], 1)], '27745201#1': [(['27745201#1'], 1)], '27745201#2': [(['27745201#2'], 1)], '27745201#3': [(['27745201#3'], 1)], '27755141#0': [(['27755141#0'], 1)], '27755141#1': [(['27755141#1'], 1)], '27755975#0': [(['27755975#0'], 1)], '27755975#1': [(['27755975#1'], 1)], '27756045': [(['27756045'], 1)], '27756067': [(['27756067'], 1)], '27756068': [(['27756068'], 1)], '277611272#0': [(['277611272#0'], 1)], '277611272#1': [(['277611272#1'], 1)], '277611272#2': [(['277611272#2'], 1)], '277611272#3': [(['277611272#3'], 1)], '277611282#0': [(['277611282#0'], 1)], '277611282#1': [(['277611282#1'], 1)], '277611282#2': [(['277611282#2'], 1)], '277611282#3': [(['277611282#3'], 1)], '277620580#0': [(['277620580#0'], 1)], '277620580#1': [(['277620580#1'], 1)], '277620580#2': [(['277620580#2'], 1)], '277620580#3': [(['277620580#3'], 1)], '277620580#4': [(['277620580#4'], 1)], '277620581': [(['277620581'], 1)], '277620582#0': [(['277620582#0'], 1)], '277620582#1': [(['277620582#1'], 1)], '277620583#0': [(['277620583#0'], 1)], '277620583#1': [(['277620583#1'], 1)], '277620584#0': [(['277620584#0'], 1)], '277620584#1': [(['277620584#1'], 1)], '277620584#2': [(['277620584#2'], 1)], '277620585#0': [(['277620585#0'], 1)], '277620585#1': [(['277620585#1'], 1)], '277620586#0': [(['277620586#0'], 1)], '277620586#1': [(['277620586#1'], 1)], '277620589#0': [(['277620589#0'], 1)], '277620589#1': [(['277620589#1'], 1)], '277620589#2': [(['277620589#2'], 1)], '277621242': [(['277621242'], 1)], '277621243': [(['277621243'], 1)], '277785529#0': [(['277785529#0'], 1)], '277785529#1': [(['277785529#1'], 1)], '278568550#0': [(['278568550#0'], 1)], '278568550#1': [(['278568550#1'], 1)], '278568550#2': [(['278568550#2'], 1)], '278568551': [(['278568551'], 1)], '279157050': [(['279157050'], 1)], '279157858': [(['279157858'], 1)], '279157860#0': [(['279157860#0'], 1)], '279157860#1': [(['279157860#1'], 1)], '279157861': [(['279157861'], 1)], '279157862#0': [(['279157862#0'], 1)], '279157862#1': [(['279157862#1'], 1)], '279157865': [(['279157865'], 1)], '279157869': [(['279157869'], 1)], '279157870#0': [(['279157870#0'], 1)], '279157870#1': [(['279157870#1'], 1)], '279157874': [(['279157874'], 1)], '279159131': [(['279159131'], 1)], '279628105': [(['279628105'], 1)], '279628107': [(['279628107'], 1)], '279661002#0': [(['279661002#0'], 1)], '279661002#1': [(['279661002#1'], 1)], '279661002#2': [(['279661002#2'], 1)], '280774527': [(['280774527'], 1)], '280775060': [(['280775060'], 1)], '284970400': [(['284970400'], 1)], '284970401': [(['284970401'], 1)], '285197070#0': [(['285197070#0'], 1)], '285197070#1': [(['285197070#1'], 1)], '285197071#0': [(['285197071#0'], 1)], '285197071#1': [(['285197071#1'], 1)], '285197072#0': [(['285197072#0'], 1)], '285197072#1': [(['285197072#1'], 1)], '285197074': [(['285197074'], 1)], '285197078': [(['285197078'], 1)], '285197079#0': [(['285197079#0'], 1)], '285197079#1': [(['285197079#1'], 1)], '285197079#2': [(['285197079#2'], 1)], '285197079#3': [(['285197079#3'], 1)], '285884450#0': [(['285884450#0'], 1)], '285884450#1': [(['285884450#1'], 1)], '286234843#0': [(['286234843#0'], 1)], '286234843#1': [(['286234843#1'], 1)], '286234843#2': [(['286234843#2'], 1)], '286234844#0': [(['286234844#0'], 1)], '286234844#1': [(['286234844#1'], 1)], '286242733#0': [(['286242733#0'], 1)], '286242733#1': [(['286242733#1'], 1)], '286564830#0': [(['286564830#0'], 1)], '286564830#1': [(['286564830#1'], 1)], '286564830#2': [(['286564830#2'], 1)], '286577466#0': [(['286577466#0'], 1)], '286577466#1': [(['286577466#1'], 1)], '286583306#0': [(['286583306#0'], 1)], '286583306#1': [(['286583306#1'], 1)], '286583306#2': [(['286583306#2'], 1)], '286585734': [(['286585734'], 1)], '28660860#0': [(['28660860#0'], 1)], '28660860#1': [(['28660860#1'], 1)], '28660861': [(['28660861'], 1)], '28661080': [(['28661080'], 1)], '28661081#0': [(['28661081#0'], 1)], '28661081#1': [(['28661081#1'], 1)], '28661081#2': [(['28661081#2'], 1)], '288311235': [(['288311235'], 1)], '288311236#0': [(['288311236#0'], 1)], '288311236#1': [(['288311236#1'], 1)], '288311237#0': [(['288311237#0'], 1)], '288311237#1': [(['288311237#1'], 1)], '288311237#2': [(['288311237#2'], 1)], '288311237#3': [(['288311237#3'], 1)], '288311237#4': [(['288311237#4'], 1)], '288313992': [(['288313992'], 1)], '288313993#0': [(['288313993#0'], 1)], '288313993#1': [(['288313993#1'], 1)], '288313993#2': [(['288313993#2'], 1)], '288931912#0': [(['288931912#0'], 1)], '288931912#1': [(['288931912#1'], 1)], '289381223#0': [(['289381223#0'], 1)], '289381223#1': [(['289381223#1'], 1)], '289381224': [(['289381224'], 1)], '289406968': [(['289406968'], 1)], '289985271#0': [(['289985271#0'], 1)], '289985271#1': [(['289985271#1'], 1)], '289985271#2': [(['289985271#2'], 1)], '291766244#0': [(['291766244#0'], 1)], '291766244#1': [(['291766244#1'], 1)], '291766245#0': [(['291766245#0'], 1)], '291766245#1': [(['291766245#1'], 1)], '291766246': [(['291766246'], 1)], '291766247#0': [(['291766247#0'], 1)], '291766247#1': [(['291766247#1'], 1)], '291766248#0': [(['291766248#0'], 1)], '291766248#1': [(['291766248#1'], 1)], '291766249#0': [(['291766249#0'], 1)], '291766249#1': [(['291766249#1'], 1)], '291766250#0': [(['291766250#0'], 1)], '291766250#1': [(['291766250#1'], 1)], '293717175#0': [(['293717175#0'], 1)], '293717175#1': [(['293717175#1'], 1)], '293717175#2': [(['293717175#2'], 1)], '297323851': [(['297323851'], 1)], '298664363': [(['298664363'], 1)], '299646741': [(['299646741'], 1)], '299735473#0': [(['299735473#0'], 1)], '299735473#1': [(['299735473#1'], 1)], '300981654': [(['300981654'], 1)], '301140399': [(['301140399'], 1)], '301140405': [(['301140405'], 1)], '301140410#0': [(['301140410#0'], 1)], '301140410#1': [(['301140410#1'], 1)], '30325860#0': [(['30325860#0'], 1)], '30325860#1': [(['30325860#1'], 1)], '30325860#2': [(['30325860#2'], 1)], '305146903': [(['305146903'], 1)], '305146904': [(['305146904'], 1)], '305863481': [(['305863481'], 1)], '307140232': [(['307140232'], 1)], '31009600#0': [(['31009600#0'], 1)], '31009600#1': [(['31009600#1'], 1)], '31009600#2': [(['31009600#2'], 1)], '31009600#3': [(['31009600#3'], 1)], '31009602#0': [(['31009602#0'], 1)], '31009602#1': [(['31009602#1'], 1)], '31009602#2': [(['31009602#2'], 1)], '31009602#3': [(['31009602#3'], 1)], '31009602#4': [(['31009602#4'], 1)], '31009602#5': [(['31009602#5'], 1)], '316592547#0': [(['316592547#0'], 1)], '316592547#1': [(['316592547#1'], 1)], '31898400': [(['31898400'], 1)], '319913619#0': [(['319913619#0'], 1)], '319913619#1': [(['319913619#1'], 1)], '324009323#0': [(['324009323#0'], 1)], '324009323#1': [(['324009323#1'], 1)], '324009323#2': [(['324009323#2'], 1)], '324269433': [(['324269433'], 1)], '325761881': [(['325761881'], 1)], '328930799': [(['328930799'], 1)], '328980176': [(['328980176'], 1)], '328983401#0': [(['328983401#0'], 1)], '328983401#1': [(['328983401#1'], 1)], '352975912': [(['352975912'], 1)], '352975915': [(['352975915'], 1)], '356628034#0': [(['356628034#0'], 1)], '356628034#1': [(['356628034#1'], 1)], '360721269#0': [(['360721269#0'], 1)], '360721269#1': [(['360721269#1'], 1)], '360721275#0': [(['360721275#0'], 1)], '360721275#1': [(['360721275#1'], 1)], '360721279': [(['360721279'], 1)], '360721283#0': [(['360721283#0'], 1)], '360721283#1': [(['360721283#1'], 1)], '371385457': [(['371385457'], 1)], '37335402#0': [(['37335402#0'], 1)], '37335402#1': [(['37335402#1'], 1)], '37335403#0': [(['37335403#0'], 1)], '37335403#1': [(['37335403#1'], 1)], '37335594#0': [(['37335594#0'], 1)], '37335594#1': [(['37335594#1'], 1)], '37338980#0': [(['37338980#0'], 1)], '37338980#1': [(['37338980#1'], 1)], '37338981#0': [(['37338981#0'], 1)], '37338981#1': [(['37338981#1'], 1)], '37634056#0': [(['37634056#0'], 1)], '37634056#1': [(['37634056#1'], 1)], '37732652#0': [(['37732652#0'], 1)], '37732652#1': [(['37732652#1'], 1)], '37732892': [(['37732892'], 1)], '379871047#0': [(['379871047#0'], 1)], '379871047#1': [(['379871047#1'], 1)], '379871048#0': [(['379871048#0'], 1)], '379871048#1': [(['379871048#1'], 1)], '379871048#2': [(['379871048#2'], 1)], '379871905#0': [(['379871905#0'], 1)], '379871905#1': [(['379871905#1'], 1)], '379872550': [(['379872550'], 1)], '379908019#0': [(['379908019#0'], 1)], '379908019#1': [(['379908019#1'], 1)], '384601626': [(['384601626'], 1)], '396221878': [(['396221878'], 1)], '396886261#0': [(['396886261#0'], 1)], '396886261#1': [(['396886261#1'], 1)], '396886261#2': [(['396886261#2'], 1)], '397339236': [(['397339236'], 1)], '400378403': [(['400378403'], 1)], '400378404': [(['400378404'], 1)], '400378406#0': [(['400378406#0'], 1)], '400378406#1': [(['400378406#1'], 1)], '400608137#0': [(['400608137#0'], 1)], '400608137#1': [(['400608137#1'], 1)], '400608141': [(['400608141'], 1)], '400608145': [(['400608145'], 1)], '400609070#0': [(['400609070#0'], 1)], '400609070#1': [(['400609070#1'], 1)], '400738632#0': [(['400738632#0'], 1)], '400738632#1': [(['400738632#1'], 1)], '400738632#2': [(['400738632#2'], 1)], '4018835#0': [(['4018835#0'], 1)], '4018835#1': [(['4018835#1'], 1)], '4018845': [(['4018845'], 1)], '404253784': [(['404253784'], 1)], '422944644#0': [(['422944644#0'], 1)], '422944644#1': [(['422944644#1'], 1)], '42453104#0': [(['42453104#0'], 1)], '42453104#1': [(['42453104#1'], 1)], '42453343#0': [(['42453343#0'], 1)], '42453343#1': [(['42453343#1'], 1)], '42453343#2': [(['42453343#2'], 1)], '42453343#3': [(['42453343#3'], 1)], '42453414#0': [(['42453414#0'], 1)], '42453414#1': [(['42453414#1'], 1)], '42453415': [(['42453415'], 1)], '42453459#0': [(['42453459#0'], 1)], '42453459#1': [(['42453459#1'], 1)], '42453459#2': [(['42453459#2'], 1)], '42453505': [(['42453505'], 1)], '42453519#0': [(['42453519#0'], 1)], '42453519#1': [(['42453519#1'], 1)], '42453519#2': [(['42453519#2'], 1)], '43015733': [(['43015733'], 1)], '43015940': [(['43015940'], 1)], '43016041#0': [(['43016041#0'], 1)], '43016041#1': [(['43016041#1'], 1)], '43061150#0': [(['43061150#0'], 1)], '43061150#1': [(['43061150#1'], 1)], '43061150#2': [(['43061150#2'], 1)], '43061361#0': [(['43061361#0'], 1)], '43061361#1': [(['43061361#1'], 1)], '43061432': [(['43061432'], 1)], '43061997': [(['43061997'], 1)], '43144476': [(['43144476'], 1)], '43160119#0': [(['43160119#0'], 1)], '43160119#1': [(['43160119#1'], 1)], '43165905#0': [(['43165905#0'], 1)], '43165905#1': [(['43165905#1'], 1)], '43165905#2': [(['43165905#2'], 1)], '43375972': [(['43375972'], 1)], '442897384#0': [(['442897384#0'], 1)], '442897384#1': [(['442897384#1'], 1)], '442897385': [(['442897385'], 1)], '443251776#0': [(['443251776#0'], 1)], '443251776#1': [(['443251776#1'], 1)], '443251777#0': [(['443251777#0'], 1)], '443251777#1': [(['443251777#1'], 1)], '443251780#0': [(['443251780#0'], 1)], '443251780#1': [(['443251780#1'], 1)], '444404990': [(['444404990'], 1)], '44447728': [(['44447728'], 1)], '444718770': [(['444718770'], 1)], '444718771#0': [(['444718771#0'], 1)], '444718771#1': [(['444718771#1'], 1)], '444883707#0': [(['444883707#0'], 1)], '444883707#1': [(['444883707#1'], 1)], '444883707#2': [(['444883707#2'], 1)], '44935513': [(['44935513'], 1)], '44935644': [(['44935644'], 1)], '44935719': [(['44935719'], 1)], '44935827#0': [(['44935827#0'], 1)], '44935827#1': [(['44935827#1'], 1)], '44935827#2': [(['44935827#2'], 1)], '44935939#0': [(['44935939#0'], 1)], '44935939#1': [(['44935939#1'], 1)], '44935988#0': [(['44935988#0'], 1)], '44935988#1': [(['44935988#1'], 1)], '4494778#0': [(['4494778#0'], 1)], '4494778#1': [(['4494778#1'], 1)], '4494778#2': [(['4494778#2'], 1)], '4519675': [(['4519675'], 1)], '4519677#0': [(['4519677#0'], 1)], '4519677#1': [(['4519677#1'], 1)], '4519678': [(['4519678'], 1)], '4520040#0': [(['4520040#0'], 1)], '4520040#1': [(['4520040#1'], 1)], '4520040#2': [(['4520040#2'], 1)], '4520043': [(['4520043'], 1)], '4520044': [(['4520044'], 1)], '4520045#0': [(['4520045#0'], 1)], '4520045#1': [(['4520045#1'], 1)], '4520047#0': [(['4520047#0'], 1)], '4520047#1': [(['4520047#1'], 1)], '4520048#0': [(['4520048#0'], 1)], '4520048#1': [(['4520048#1'], 1)], '4520049#0': [(['4520049#0'], 1)], '4520049#1': [(['4520049#1'], 1)], '4520054#0': [(['4520054#0'], 1)], '4520054#1': [(['4520054#1'], 1)], '4520054#2': [(['4520054#2'], 1)], '4520055#0': [(['4520055#0'], 1)], '4520055#1': [(['4520055#1'], 1)], '4520061#0': [(['4520061#0'], 1)], '4520061#1': [(['4520061#1'], 1)], '4520062#0': [(['4520062#0'], 1)], '4520062#1': [(['4520062#1'], 1)], '4520063#0': [(['4520063#0'], 1)], '4520063#1': [(['4520063#1'], 1)], '4520068#0': [(['4520068#0'], 1)], '4520068#1': [(['4520068#1'], 1)], '4520069': [(['4520069'], 1)], '4520071#0': [(['4520071#0'], 1)], '4520071#1': [(['4520071#1'], 1)], '4520072#0': [(['4520072#0'], 1)], '4520072#1': [(['4520072#1'], 1)], '4520072#2': [(['4520072#2'], 1)], '4520073#0': [(['4520073#0'], 1)], '4520073#1': [(['4520073#1'], 1)], '4520073#2': [(['4520073#2'], 1)], '4520074#0': [(['4520074#0'], 1)], '4520074#1': [(['4520074#1'], 1)], '4520074#2': [(['4520074#2'], 1)], '4520075#0': [(['4520075#0'], 1)], '4520075#1': [(['4520075#1'], 1)], '4520075#2': [(['4520075#2'], 1)], '4520076#0': [(['4520076#0'], 1)], '4520076#1': [(['4520076#1'], 1)], '4520076#2': [(['4520076#2'], 1)], '4520076#3': [(['4520076#3'], 1)], '4520077#0': [(['4520077#0'], 1)], '4520077#1': [(['4520077#1'], 1)], '4520077#2': [(['4520077#2'], 1)], '4520077#3': [(['4520077#3'], 1)], '4520078#0': [(['4520078#0'], 1)], '4520078#1': [(['4520078#1'], 1)], '4520078#2': [(['4520078#2'], 1)], '4520080': [(['4520080'], 1)], '4520081#0': [(['4520081#0'], 1)], '4520081#1': [(['4520081#1'], 1)], '4520082#0': [(['4520082#0'], 1)], '4520082#1': [(['4520082#1'], 1)], '4520082#2': [(['4520082#2'], 1)], '4520083#0': [(['4520083#0'], 1)], '4520083#1': [(['4520083#1'], 1)], '4520083#2': [(['4520083#2'], 1)], '4520100#0': [(['4520100#0'], 1)], '4520100#1': [(['4520100#1'], 1)], '4520101#0': [(['4520101#0'], 1)], '4520101#1': [(['4520101#1'], 1)], '4520101#2': [(['4520101#2'], 1)], '4520102#0': [(['4520102#0'], 1)], '4520102#1': [(['4520102#1'], 1)], '4520105': [(['4520105'], 1)], '45671217#0': [(['45671217#0'], 1)], '45671217#1': [(['45671217#1'], 1)], '45671218#0': [(['45671218#0'], 1)], '45671218#1': [(['45671218#1'], 1)], '45671218#2': [(['45671218#2'], 1)], '45671218#3': [(['45671218#3'], 1)], '45671218#4': [(['45671218#4'], 1)], '45671218#5': [(['45671218#5'], 1)], '45671218#6': [(['45671218#6'], 1)], '464868336#0': [(['464868336#0'], 1)], '464868336#1': [(['464868336#1'], 1)], '465958707': [(['465958707'], 1)], '472195951': [(['472195951'], 1)], '472195958': [(['472195958'], 1)], '4868903#0': [(['4868903#0'], 1)], '4868903#1': [(['4868903#1'], 1)], '4868904#0': [(['4868904#0'], 1)], '4868904#1': [(['4868904#1'], 1)], '4868905#0': [(['4868905#0'], 1)], '4868905#1': [(['4868905#1'], 1)], '4868918': [(['4868918'], 1)], '4868920#0': [(['4868920#0'], 1)], '4868920#1': [(['4868920#1'], 1)], '4868920#2': [(['4868920#2'], 1)], '4868921': [(['4868921'], 1)], '4868922#0': [(['4868922#0'], 1)], '4868922#1': [(['4868922#1'], 1)], '4868922#2': [(['4868922#2'], 1)], '4868922#3': [(['4868922#3'], 1)], '4868922#4': [(['4868922#4'], 1)], '4868924': [(['4868924'], 1)], '4868925': [(['4868925'], 1)], '4868926': [(['4868926'], 1)], '4868927': [(['4868927'], 1)], '4868928#0': [(['4868928#0'], 1)], '4868928#1': [(['4868928#1'], 1)], '4868928#2': [(['4868928#2'], 1)], '4868928#3': [(['4868928#3'], 1)], '4868928#4': [(['4868928#4'], 1)], '491530383': [(['491530383'], 1)], '492314001#0': [(['492314001#0'], 1)], '492314001#1': [(['492314001#1'], 1)], '492314001#2': [(['492314001#2'], 1)], '5110264': [(['5110264'], 1)], '5110266': [(['5110266'], 1)], '5110269#0': [(['5110269#0'], 1)], '5110269#1': [(['5110269#1'], 1)], '5110272#0': [(['5110272#0'], 1)], '5110272#1': [(['5110272#1'], 1)], '5110272#2': [(['5110272#2'], 1)], '5110272#3': [(['5110272#3'], 1)], '515460394': [(['515460394'], 1)], '51855672#0': [(['51855672#0'], 1)], '51855672#1': [(['51855672#1'], 1)], '52788949#0': [(['52788949#0'], 1)], '52788949#1': [(['52788949#1'], 1)], '52788949#2': [(['52788949#2'], 1)], '529192417#0': [(['529192417#0'], 1)], '529192417#1': [(['529192417#1'], 1)], '529192419#0': [(['529192419#0'], 1)], '529192419#1': [(['529192419#1'], 1)], '53869268': [(['53869268'], 1)], '53869270': [(['53869270'], 1)], '53896956': [(['53896956'], 1)], '53896958': [(['53896958'], 1)], '54643434': [(['54643434'], 1)], '56314379': [(['56314379'], 1)], '56935336': [(['56935336'], 1)], '57742680#0': [(['57742680#0'], 1)], '57742680#1': [(['57742680#1'], 1)], '58666560#0': [(['58666560#0'], 1)], '58666560#1': [(['58666560#1'], 1)], '58666560#2': [(['58666560#2'], 1)], '58666560#3': [(['58666560#3'], 1)], '586824803#0': [(['586824803#0'], 1)], '586824803#1': [(['586824803#1'], 1)], '587478231': [(['587478231'], 1)], '598184970#0': [(['598184970#0'], 1)], '598184970#1': [(['598184970#1'], 1)], '598184970#2': [(['598184970#2'], 1)], '598535494': [(['598535494'], 1)], '60005025#0': [(['60005025#0'], 1)], '60005025#1': [(['60005025#1'], 1)], '60005026': [(['60005026'], 1)], '614040413': [(['614040413'], 1)], '618591059': [(['618591059'], 1)], '621957866': [(['621957866'], 1)], '64343117#0': [(['64343117#0'], 1)], '64343117#1': [(['64343117#1'], 1)], '65617783#0': [(['65617783#0'], 1)], '65617783#1': [(['65617783#1'], 1)], '65617783#2': [(['65617783#2'], 1)], '682256303#0': [(['682256303#0'], 1)], '682256303#1': [(['682256303#1'], 1)], '79673984#0': [(['79673984#0'], 1)], '79673984#1': [(['79673984#1'], 1)], '79673985': [(['79673985'], 1)], '7980549': [(['7980549'], 1)], '7980550#0': [(['7980550#0'], 1)], '7980550#1': [(['7980550#1'], 1)], '8132442#0': [(['8132442#0'], 1)], '8132442#1': [(['8132442#1'], 1)], '8132446#0': [(['8132446#0'], 1)], '8132446#1': [(['8132446#1'], 1)], '8132447#0': [(['8132447#0'], 1)], '8132447#1': [(['8132447#1'], 1)], '8311367#0': [(['8311367#0'], 1)], '8311367#1': [(['8311367#1'], 1)], '8311367#2': [(['8311367#2'], 1)], '8311371#0': [(['8311371#0'], 1)], '8311371#1': [(['8311371#1'], 1)], '8311378': [(['8311378'], 1)], '8311387#0': [(['8311387#0'], 1)], '8311387#1': [(['8311387#1'], 1)], '8311387#2': [(['8311387#2'], 1)], '8311387#3': [(['8311387#3'], 1)], '8311400': [(['8311400'], 1)], '83605155#0': [(['83605155#0'], 1)], '83605155#1': [(['83605155#1'], 1)], '8423864#0': [(['8423864#0'], 1)], '8423864#1': [(['8423864#1'], 1)], '8423864#2': [(['8423864#2'], 1)], '8423865': [(['8423865'], 1)], '8423867': [(['8423867'], 1)], '8432342#0': [(['8432342#0'], 1)], '8432342#1': [(['8432342#1'], 1)], '8432343#0': [(['8432343#0'], 1)], '8432343#1': [(['8432343#1'], 1)], '86059925': [(['86059925'], 1)]}
        }

### 2.3 Rerunning the Simulation

We are now ready to rerun the simulation with fully defined vehicle routes and a limited number of traversable edges. If we run the cell below, we can see the new simulation in action.

In [ ]:
flow_params = dict(
    exp_tag='bay_bridge',
    env_name=TestEnv,
    network=BayBridgeOSMNetwork,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=new_initial_config,
)

# number of time steps
flow_params['env'].horizon = 100000000
exp = Experiment(flow_params)

# run the sumo simulation
_ = exp.run(1)

## 3. Other Tips

This tutorial introduces how to incorporate OpenStreetMap files in Flow. This feature, however, does not negate other features that are introduced in other tutorials and documentation. For example, if you would like to not have vehicles be originated side-by-side within a network, this can still be done by specifying a "random" spacing for vehicles as follows:

    initial_config = InitialConfig(
        spacing="random",
        edges_distribution=EDGES_DISTRIBUTION
    )

In addition, inflows of vehicles can be added to networks imported from OpenStreetMap as they are for any other network (see the tutorial on adding inflows for more on this).